In [1]:
# 1. 필수 라이브러리 설치
# (Colab 세션이 새로 시작될 때마다 설치해야 합니다)
!pip install -q transformers datasets accelerate pandas

import os
import pandas as pd
from datasets import load_dataset
from google.colab import drive

In [2]:
# 2. 구글 드라이브 마운트 (창고 연결)
print("🔑 구글 드라이브 연결 중...")
drive.mount('/content/drive')

🔑 구글 드라이브 연결 중...
Mounted at /content/drive


In [3]:
# 3. 프로젝트 경로 설정 (구글 드라이브 내 폴더)
# 이 경로에 데이터와 나중에 만들 모델이 저장됩니다.
project_path = '/content/drive/MyDrive/News_Summary_Project'
data_file_path = os.path.join(project_path, 'news_train.csv')

# 폴더가 없으면 생성
if not os.path.exists(project_path):
    os.makedirs(project_path)
    print(f"📂 드라이브에 새 폴더 생성 완료: {project_path}")
else:
    print(f"📂 드라이브 폴더 확인 완료: {project_path}")

📂 드라이브 폴더 확인 완료: /content/drive/MyDrive/News_Summary_Project


In [4]:
# 4. 데이터 로드 로직 (있으면 읽고, 없으면 다운로드)
if os.path.exists(data_file_path):
    print(f"\n⚡ 드라이브에 저장된 데이터가 있습니다. 바로 불러옵니다.")
    economy_df = pd.read_csv(data_file_path)
else:
    print(f"\n⬇️ 저장된 데이터가 없습니다. 새로 다운로드하고 정제합니다...")

    # 데이터셋 다운로드 (daekeun-ml/naver-news-summarization-ko)
    try:
        dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
        full_df = pd.DataFrame(dataset['train'])

        # 'economy' 또는 '경제' 카테고리 필터링 (10,000개 샘플링)
        # 데이터셋 컬럼 확인 후 필터링
        if 'category' in full_df.columns:
            # 카테고리가 'economy' 혹은 '경제'인 것 찾기
            target_category = ['economy', '경제']
            economy_df = full_df[full_df['category'].isin(target_category)].sample(n=10000, random_state=42)
        else:
            # 카테고리가 없으면 전체에서 랜덤 추출
            print("⚠️ 카테고리 정보 없음. 전체에서 랜덤 추출합니다.")
            economy_df = full_df.sample(n=10000, random_state=42)

        # 드라이브에 CSV로 영구 저장
        economy_df.to_csv(data_file_path, index=False)
        print(f"✅ 데이터 정제 후 드라이브에 저장 완료: {data_file_path}")

    except Exception as e:
        print(f"❌ 데이터 다운로드 실패: {e}")
        raise e


⚡ 드라이브에 저장된 데이터가 있습니다. 바로 불러옵니다.


In [5]:
# 5. 최종 데이터 확인
print("\n--- 👀 데이터 확인 (상위 1개) ---")
print(f"제목: {economy_df.iloc[0]['title']}")
print(f"요약: {economy_df.iloc[0]['summary']}")
print(f"데이터 개수: {len(economy_df)}개")


--- 👀 데이터 확인 (상위 1개) ---
제목: 차세대 반도체 SiC 개발 성공 글로벌 경쟁력 확보
요약: 인텔의 팻 갤싱어 인텔 최고경영자 CEO는 올해 초 내연 차량이 전기차 자율차 등으로 진화할수록 반도체의 쓰임새가 늘어난다며 차량용 반도체 시장 진출을 알렸고 오랜 기간 내공을 갈고 닦은 알짜 기업들이 러브콜을 받는 배경 중 하나로 아이에이가 전력 반도체 중 특히 집중하는 종류는 IGBT 절연 게이트 양극성 트랜지스터다.
데이터 개수: 10000개


In [6]:
from transformers import PreTrainedTokenizerFast
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch

In [7]:
# 1. 토크나이저 로드 (SKT KoGPT2)
# bos(문장시작), eos(문장끝), unk(모름), pad(길이맞춤), mask(가림) 토큰을 명시합니다.
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)

print(f"✅ 토크나이저 로드 완료! (단어 집합 크기: {len(tokenizer)})")

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


✅ 토크나이저 로드 완료! (단어 집합 크기: 51200)


In [8]:
# 2. 커스텀 데이터셋 클래스 정의
# 데이터프레임의 텍스트를 모델에 넣을 수 있는 형태로 변환하는 공장입니다.
class NewsSummaryDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.data = df
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        title = row['title']
        text = row['document'] # 뉴스 본문
        summary = row['summary'] # 정답 요약문

        # [중요] 학습용 프롬프트 형식 정의
        # 뉴스를 보여주고 요약을 뱉어내도록 유도하는 문자열 포맷
        # 너무 길면 잘라내기 위해 본문은 앞 300자 정도만 사용 (메모리 절약 및 핵심 파악)
        input_text = f"### 뉴스: {title} \n {text[:300]} \n ### 요약: {summary}" + tokenizer.eos_token

        # 토크나이징 (텍스트 -> 숫자)
        inputs = self.tokenizer(
            input_text,
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        # GPT 학습의 핵심: 입력(input_ids)이 곧 정답(labels)입니다. (Next Token Prediction)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': input_ids
        }

In [9]:
# 3. 학습용(Train) / 검증용(Validation) 데이터 분리
# 과적합 확인을 위해 10%는 떼어놓습니다.
train_df, val_df = train_test_split(economy_df, test_size=0.1, random_state=42)

print(f"학습 데이터: {len(train_df)}개 / 검증 데이터: {len(val_df)}개")

학습 데이터: 9000개 / 검증 데이터: 1000개


In [10]:
# 4. 데이터셋 인스턴스 생성
train_dataset = NewsSummaryDataset(train_df, tokenizer)
val_dataset = NewsSummaryDataset(val_df, tokenizer)

# --- 👀 잘 만들어졌는지 1개만 까서 확인해보자 ---
print("\n--- 🔍 데이터셋 샘플 확인 (디코딩 결과) ---")
sample = train_dataset[0]
print(tokenizer.decode(sample['input_ids'], skip_special_tokens=False))
print("\n✅ 데이터셋 준비가 완료되었습니다.")


--- 🔍 데이터셋 샘플 확인 (디코딩 결과) ---
### 뉴스: 부채비율 3분의 1로... CGV·롯데시네마 재무구조 개선 속도 
 CGV 4000억원 영구 전환사채 발행 부채비율 1944%서 603%로 채무 상환 및 ‘고급 특별상영관’ 확충 롯데시네마도 영구채 발행 확대... “자금 조달로 부채비율 개선” 신종 코로나바이러스 감염증 코로나19 으로 경영난에 시달렸던 영화관 사업자들이 리오프닝 경제활동 재개 이후 자본 확충을 통한 재무구조 개선에 나서고 있다. 지난 2년간 폭등한 부채비율을 낮추고 고급화·개인화된 고객 수요를 충족하기 위해 고급·특별관에 투자할 자금을 확보하기 위해서다. 그래픽 이은현 3일 금융감독원에 따르면 CJ CGV 079160 는 이달 4 
 ### 요약: 코로나19 으로 경영난에 시달렸던 영화관 사업자들이 리오프닝 경제활동 재개 이후 부채비율을 낮추고 고급·특별관에 투자할 자금을 확보하기 위해 4000억원 규모의 영구 전환사채 CB 를 발행하는 등 자본 확충을 통한 재무구조 개선에 나서고 있다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [11]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [12]:
# 1. 모델 로드 (Pre-trained Model)
# 토크나이저와 짝이 맞는 SKT KoGPT2 모델을 불러옵니다.
print("⏳ 모델을 다운로드하고 로드합니다...")
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.to("cuda") # GPU로 이동 (필수!)

⏳ 모델을 다운로드하고 로드합니다...


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [14]:
# 2. 학습 설정 (TrainingArguments) - 여기가 제일 중요!
# 구글 드라이브 경로에 저장해야 날아가지 않습니다.
output_dir = '/content/drive/MyDrive/News_Summary_Project/output'

training_args = TrainingArguments(
    output_dir=output_dir,          # 저장될 경로
    num_train_epochs=3,             # 학습 반복 횟수 (3번이면 충분)
    per_device_train_batch_size=8,  # 한 번에 학습할 데이터 양 (GPU 메모리에 맞춰 조절)
    per_device_eval_batch_size=8,   # 검증할 때 데이터 양
    logging_dir='./logs',           # 로그 저장 경로
    logging_steps=100,              # 100 step마다 로그 출력 (Loss 확인용)
    save_strategy="epoch",          # 1 epoch 끝날 때마다 임시 저장
    eval_strategy="epoch",    # 1 epoch 끝날 때마다 검증
    learning_rate=5e-5,             # 학습률 (너무 크면 못 배우고, 너무 작으면 느림)
    weight_decay=0.01,              # 과적합 방지용 규제
    warmup_steps=200,               # 초반에 천천히 학습 시작 (안정성)
    save_total_limit=2,             # 용량 아끼기 위해 최근 2개 모델만 남김
)

In [15]:
# 3. 트레이너(Trainer) 정의
# 선생님 역할을 하는 객체입니다. 모델, 설정, 데이터를 다 넘겨줍니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # 데이터가 입력될 때 배치를 예쁘게 묶어주는 역할 (기본값 사용)
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [16]:
# 4. 학습 시작! (Start Training)
print("🚀 학습을 시작합니다! (약 20~40분 소요 예정)")
trainer.train()

# 5. 최종 모델 저장 (가장 중요)
# 학습이 끝나면 최종 결과물을 드라이브에 'final_model'이라는 이름으로 저장합니다.
final_path = '/content/drive/MyDrive/News_Summary_Project/final_model'
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

print(f"\n🎉 학습 완료! 모델이 구글 드라이브에 저장되었습니다: {final_path}")

🚀 학습을 시작합니다! (약 20~40분 소요 예정)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.125600,2.010113
2,1.782300,1.885731
3,1.565000,1.849099



🎉 학습 완료! 모델이 구글 드라이브에 저장되었습니다: /content/drive/MyDrive/News_Summary_Project/final_model


In [19]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import torch

# 1. 저장된 모델 불러오기 (구글 드라이브)
save_path = '/content/drive/MyDrive/News_Summary_Project/final_model'
print(f"📂 저장된 모델을 불러옵니다: {save_path}")

try:
    model = GPT2LMHeadModel.from_pretrained(save_path)
    tokenizer = PreTrainedTokenizerFast.from_pretrained(save_path,
                                                        bos_token='</s>',
                                                        eos_token='</s>',
                                                        unk_token='<unk>',
                                                        pad_token='<pad>',
                                                        mask_token='<mask>')
    print("✅ 모델 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    # 혹시 로드 실패하면 코랩 세션에 있는 걸 바로 쓰도록 처리
    print("메모리에 있는 모델을 그대로 사용합니다.")

# GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

📂 저장된 모델을 불러옵니다: /content/drive/MyDrive/News_Summary_Project/final_model
✅ 모델 로드 성공!


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [20]:
# 2. 요약 생성 함수 정의
def summarize_news(news_text):
    # 학습 때와 똑같은 형식으로 입력을 만들어줍니다.
    input_text = f"### 뉴스: {news_text} \n ### 요약:"

    # 텍스트 -> 숫자 (토큰화)
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # 모델에게 뒷내용 생성 요청
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=512,              # 최대 길이
            num_beams=3,                 # 3개의 길을 찾아보고 가장 좋은 것 선택 (Beam Search)
            early_stopping=True,         # <eos> 나오면 즉시 종료
            no_repeat_ngram_size=3,      # 같은 말 3번 반복 금지
            repetition_penalty=1.2,      # 반복 억제 페널티
            eos_token_id=tokenizer.eos_token_id # 문장 종료 토큰 지정
        )

    # 숫자 -> 텍스트 (디코딩)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 프롬프트 부분 제거하고 요약만 추출
    summary = generated_text.split("### 요약:")[-1].strip()
    return summary

In [22]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import torch

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. 토크나이저 로드 (공통 사용)
# 저장된 경로에서 불러옵니다.
save_path = '/content/drive/MyDrive/News_Summary_Project/final_model'
tokenizer = PreTrainedTokenizerFast.from_pretrained(save_path,
                                                    bos_token='</s>', eos_token='</s>',
                                                    unk_token='<unk>', pad_token='<pad>', mask_token='<mask噠>')

# 2. 두 모델 불러오기 (비교용)
print("⏳ 모델 로딩 중... (메모리 확보를 위해 잠시 시간이 걸립니다)")

# (1) 순정 모델 (Base Model) - 학습 전
original_model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2").to(device)

# (2) 튜닝된 모델 (Fine-tuned Model) - 학습 후
finetuned_model = GPT2LMHeadModel.from_pretrained(save_path).to(device)

print("✅ 두 모델 로드 완료! 대결을 시작합니다.\n")

# 3. 비교 함수 정의
def compare_models(news_text):
    # 학습 때 썼던 프롬프트 형식 그대로 적용
    input_text = f"### 뉴스: {news_text} \n ### 요약:"
    tokenized_text = tokenizer(input_text, return_tensors="pt").to(device)

    # 생성 옵션 설정 (동일한 조건)
    gen_params = {
        "max_length": 512,
        "num_beams": 3,
        "early_stopping": True,
        "no_repeat_ngram_size": 3,
        "eos_token_id": tokenizer.eos_token_id
    }

    print(f"📰 [입력 뉴스 본문] (일부 생략)\n{news_text[:150]}...\n")
    print("-" * 50)

    # --- A. 학습 전 (Original) ---
    with torch.no_grad():
        out_origin = original_model.generate(tokenized_text['input_ids'], **gen_params)
    decoded_origin = tokenizer.decode(out_origin[0], skip_special_tokens=True)
    # 프롬프트 제거하고 순수 생성물만 추출
    res_origin = decoded_origin.split("### 요약:")[-1].strip()

    print(f"❌ [Before] 학습 전 모델의 답변:\n{res_origin}")
    print("-" * 50)

    # --- B. 학습 후 (Fine-tuned) ---
    with torch.no_grad():
        out_finetuned = finetuned_model.generate(tokenized_text['input_ids'], **gen_params)
    decoded_finetuned = tokenizer.decode(out_finetuned[0], skip_special_tokens=True)
    res_finetuned = decoded_finetuned.split("### 요약:")[-1].strip()

    print(f"⭕ [After] 파인튜닝 모델의 답변:\n{res_finetuned}")
    print("=" * 50)

# 4. 실제 테스트 데이터 입력
test_news_1 = """
미국 연방준비제도(Fed)가 기준금리를 동결했다. 제롬 파월 의장은 물가 상승률이 목표치인 2%로 내려가고 있다는 확신이 들 때까지 금리 인하는 없을 것이라고 못 박았다.
시장에서는 연내 금리 인하 기대감이 있었으나, 파월 의장의 매파적 발언으로 인해 뉴욕 증시는 일제히 하락 마감했다.
특히 기술주 중심의 나스닥 지수는 1.5% 넘게 떨어지며 큰 낙폭을 보였다.
"""

compare_models(test_news_1)

# 사용자 입력 테스트
user_input = input("\n직접 테스트할 뉴스를 입력하세요 (엔터): ")
if user_input:
    compare_models(user_input)

⏳ 모델 로딩 중... (메모리 확보를 위해 잠시 시간이 걸립니다)
✅ 두 모델 로드 완료! 대결을 시작합니다.

📰 [입력 뉴스 본문] (일부 생략)

미국 연방준비제도(Fed)가 기준금리를 동결했다. 제롬 파월 의장은 물가 상승률이 목표치인 2%로 내려가고 있다는 확신이 들 때까지 금리 인하는 없을 것이라고 못 박았다. 
시장에서는 연내 금리 인하 기대감이 있었으나, 파월 의장의 매파적 발언으로 인해 뉴욕 증시는 ...

--------------------------------------------------
❌ [Before] 학습 전 모델의 답변:
# #
## 미국 경제지표 # #20180904 #미롱_식단 
점심 : #다이어트식단
#diet #eatclean #식단조절 #건강식 #저칼로리 #고지방 #닭가슴살 #샐러드 #오메가3 #단백질 #당충전 #지방간 #고지혈증 #콜레스테롤 #간식스타그램 #홈쿡 #인스타푸드 #온더테이블 #집밥 #요리스타그램 ♀ #아침 #저녁메뉴 #치즈케이크 #카페 #디저트 #빵 #먹스타 #맛있다
#먹부림 #yummy #dessert #instafood #foodstagram #소통 #좋아요
#팔로우 #맞팔 #선팔하면맞팔
#followme #like4likes #cafe #meow #냥스타그램
#cat #sonyu #scottishfold #neko #katze ᄏᄏᄏ #이태원맛집 #신촌맛집
이태원에서 가장 유명한 맛집 중 하나인
신촌역 맛집 신촌역맛집 신촌을 소개해드릴게요!
신촌역에 위치한 신촌맛집은
신촌을 대표하는 맛집이라고 할 수 있죠?
신촌역에서 가장 유명한 곳 중 하나인 신촌의 맛집
신촌은 신촌의 대표 맛집이라 할 수 있는데요.
신촌역은 신촌의 대표적인 맛집으로
신촌의 대표 메뉴인 치즈케익이 유명한 곳
--------------------------------------------------
⭕ [After] 파인튜닝 모델의 답변:
미국 연방준비제도 Fed가 연방공개시장위원회 FOMC 회의록에서 물가 상승률 목표치를 2%로